In [20]:
import re
from itertools import count
from dateutil.parser import parse
from pytz import timezone
import numpy as np
import pandas as pd
from nameparser import HumanName

In [ ]:
def parse_raw_email_data():    
    cols_of_interest = [ 'Id', 'MetadataTo', 'MetadataFrom', 'MetadataDateSent',                        
                        'ExtractedSubject', 'ExtractedBodyText']
    rename_dict = {
        'Id': 'id',
        'MetadataTo': 'to',
        'MetadataFrom': 'from',
        'MetadataDateSent': 'date',
        'ExtractedSubject': 'subject',
        'ExtractedBodyText': 'body'
    }
    df = (pd.read_csv('Hillary_Emails.csv')
             .loc[:, cols_of_interest]
             .rename(columns=rename_dict)
             .dropna())
    return df

In [256]:
all_emails = pd.read_csv('Hillary_Emails.csv')

In [257]:
all_emails.columns

Index(['Id', 'DocNumber', 'MetadataSubject', 'MetadataTo', 'MetadataFrom',
       'SenderPersonId', 'MetadataDateSent', 'MetadataDateReleased',
       'MetadataPdfLink', 'MetadataCaseNumber', 'MetadataDocumentClass',
       'ExtractedSubject', 'ExtractedTo', 'ExtractedFrom', 'ExtractedCc',
       'ExtractedDateSent', 'ExtractedCaseNumber', 'ExtractedDocNumber',
       'ExtractedDateReleased', 'ExtractedReleaseInPartOrFull',
       'ExtractedBodyText', 'RawText'],
      dtype='object')

In [21]:
tos = all_emails[['MetadataTo', 'ExtractedTo', 'ExtractedCc']]

In [303]:
import fuzzywuzzy

In [353]:
metadata_to = pd.DataFrame(all_emails['MetadataTo'].dropna().unique(), columns=['MetadataTo'])
metadata_to_emails = metadata_to[metadata_to['MetadataTo'].str.contains("@")]
emails_to_match = list(fuzzywuzzy.process.dedupe(metadata_to_emails.MetadataTo.str.lower(), threshold=90))
metadata_to_names = metadata_to[~metadata_to['MetadataTo'].str.contains("@")]
metadata_to_names['MetadataTo'] = metadata_to_names.MetadataTo.apply(lambda s: s.replace('.', ' '))
metadata_to_names['FirstName'] = metadata_to_names.MetadataTo.apply(lambda s: HumanName(s).first)
metadata_to_names['MiddleName'] = metadata_to_names.MetadataTo.apply(lambda s: HumanName(s).middle)
metadata_to_names['LastName'] = metadata_to_names.MetadataTo.apply(lambda s: HumanName(s).last)
metadata_to_names['email_address'] = ''

c:\users\danie\desktop\w210\venv_docs\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\danie\desktop\w210\venv_docs\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\danie\desktop\w210\venv_docs\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydat

In [357]:
for email in emails_to_match:
    for i, row in metadata_to_names.iterrows():
        if (len(row.LastName) > 1) & (row.LastName.lower() in email):
            if (len(row.FirstName) > 0) & (row.FirstName[0].lower() in email):
                row['email_address'] = email
        elif (row.MetadataTo.lower() in email):
            row['email_address'] = email

In [356]:
for idx, r in metadata_to_names.iterrows():
    for i, row in metadata_to_names.iterrows():
        if (len(row.LastName) > 1) & (row.LastName.lower() in r.MetadataTo.lower()) & (' ' not in r.MetadataTo.lower()):
            if (len(row.FirstName) > 0):
                if row.FirstName.lower()[0] in r.MetadataTo.lower():
                    r['FirstName'] = row.FirstName
                    r['LastName'] = row.LastName

In [344]:
for idx, row in metadata_to_names.iterrows():
    if ~(" " in row['MetadataTo']) & (row['MetadataTo'].islower()) & (row['LastName'] == ''):
        row['MetadataTo'] = row['MetadataTo'][0] + ' ' + row['MetadataTo'][1:]

In [271]:
def split_names(string):
    if ~(string.str.contains(" ")) & (string.str.islower()):
        string = string[0] + ' ' + string[1:]

In [358]:
metadata_to_names

,MetadataTo,FirstName,MiddleName,LastName,email_address
0,H,H,,,schwerindb@state.gov
1,;H,;H,,,
2,"Abedin, Huma",Huma,,Abedin,
4,"Sherman, Wendy R",Wendy,R,Sherman,
6,"Mills, Cheryl D",Cheryl,D,Mills,cheryl.mills@
7,"Flores, Oscar",Oscar,,Flores,
11,"Schwerin, Daniel B",Daniel,B,Schwerin,schwerindb@state.gov
12,"Nides, Thomas R",Thomas,R,Nides,nidestr@state.gov
13,"Sullivan, Jacob J",Jacob,J,Sullivan,jake.sullivan@
18,Jake Sullivan,Jake,,Sullivan,jake.sullivan@


In [359]:
matched_emails = metadata_to_names.email_address.to_list()

In [360]:
[value for value in emails_to_match if value not in matched_emails] 

['russorv@state.gov',
 'sulllivanjj@state.gov',
 'abendinh@state.gov',
 'millscd@state.gov',
 'hanleymr@state.gov',
 'valmmorolj@state.gov',
 'burnswj@state.gov',
 'bam@mikulski.senate.gov',
 'steinbergjb@state.gov',
 'muscantinel@state.gov',
 'cholletdh@state.gov',
 'shapiroa@state.gov',
 'david_garten@lautenberg.senate.gov',
 'verveerms@state.gov',
 'fuchsmh@state.gov',
 'campbellkm@state.gov',
 'rooneym@state.gov',
 'kellyc@state.gov',
 'holbrookerc@state.gov',
 'gordonph@state.gov',
 'rodriguezme@state.gov',
 'tauschereo@state.gov',
 'carsonj@state.gov',
 'shannonta@state.gov',
 'marshallcp@state.gov',
 'hr15@mycingular.blackberry.net',
 'hormatsrd@state.gov',
 'sterntd@state.gov',
 'brimmere@state.gov',
 'valenzuelaaa@state.gov',
 'tillemannts@state.gov']

In [370]:
metadata_to_names[metadata_to_names.MetadataTo.str.lower().str.contains('musc')]

,MetadataTo,FirstName,MiddleName,LastName,email_address
46,Imuscatine,Imuscatine,,,
47,lmuscatine,Lissa,,Muscatine,
186,"Muscatine, Lissa",Lissa,,Muscatine,
